In [4]:
import pandas as pd

#création de 4 df
df_amazon=pd.read_csv('amazon_cleaned_reviews.csv')
df_cdiscount=pd.read_csv('cdiscount_cleaned_reviews.csv')
df_rakuten=pd.read_csv('rakuten_cleaned_reviews.csv')
df_rdc=pd.read_csv('rdc_cleaned_reviews.csv')




In [6]:
# garder les lignes sans NA pour dataframe rdc
df_rdc = df_rdc[df_rdc["date"].notna()]

In [10]:
def fill_list(n, str_value):
    return [str_value]*n

In [14]:
df_amazon["plate_forme"] = fill_list(len(df_amazon), "AMAZON")
df_cdiscount["plate_forme"] = fill_list(len(df_cdiscount), "CDISCOUNT")
df_rakuten["plate_forme"] = fill_list(len(df_rakuten), "RAKUTEN")
df_rdc["plate_forme"] = fill_list(len(df_rdc), "RDC")

In [34]:
df = pd.DataFrame()
df = pd.concat([df_amazon, df_cdiscount, df_rakuten, df_rdc], axis=0)

In [36]:
df.head()

,title,rating,date,body,sentiment,category,month_year,satisfied,plate_forme
0,Exploitation car blocage de remboursement,1,2024-06-28 00:00:00,"entre les colis jetés n'importe où, et l'argen...",Negative,Shipping,2024-06,0,AMAZON
1,J'ai commandé une imprimante par…,1,2024-06-29 00:00:00,J'ai commandé une imprimante par Amazon: Je n'...,Neutral,Electronics,2024-06,0,AMAZON
2,"Colis Privé, société pas sérieuse",1,2024-06-26 00:00:00,"La société Colis Privé n'est pas sérieuse, cel...",Negative,Shipping,2024-06,0,AMAZON
3,Que se passe-t’il chez Amazon ???,1,2024-06-14 00:00:00,Que se passe-t’il chez Amazon ???\nJ’ai comman...,Neutral,Marketplace,2024-06,0,AMAZON
4,Attention!!!!,1,2024-06-28 00:00:00,Attention! Commande passée pour le portugal… d...,Neutral,Customer Service,2024-06,0,AMAZON


In [38]:
df.reindex(columns = ['plate_forme', 'title', 'rating','date','body','sentiment','category','month_year','satisfied'])

,plate_forme,title,rating,date,body,sentiment,category,month_year,satisfied
0,AMAZON,Exploitation car blocage de remboursement,1,2024-06-28 00:00:00,"entre les colis jetés n'importe où, et l'argen...",Negative,Shipping,2024-06,0
1,AMAZON,J'ai commandé une imprimante par…,1,2024-06-29 00:00:00,J'ai commandé une imprimante par Amazon: Je n'...,Neutral,Electronics,2024-06,0
2,AMAZON,"Colis Privé, société pas sérieuse",1,2024-06-26 00:00:00,"La société Colis Privé n'est pas sérieuse, cel...",Negative,Shipping,2024-06,0
3,AMAZON,Que se passe-t’il chez Amazon ???,1,2024-06-14 00:00:00,Que se passe-t’il chez Amazon ???\nJ’ai comman...,Neutral,Marketplace,2024-06,0
4,AMAZON,Attention!!!!,1,2024-06-28 00:00:00,Attention! Commande passée pour le portugal… d...,Neutral,Customer Service,2024-06,0
...,...,...,...,...,...,...,...,...,...
3191,RDC,je mets 1 étoile car il n'est possible…,1,2018-06-19 00:00:00,je mets 1 étoile car il n'est possible d'en me...,Negative,Electronics,2018-06,0
3192,RDC,"Site moyen, Service Qualité bon",2,2018-06-18 19:45:45,"J'ai commandé le 24 mai, deux casques Philips ...",Neutral,Electronics,2018-06,0
3193,RDC,voleurs!!!,1,2018-06-18 11:08:26,"bonjour, voila, j'ai effectué un achat d'une p...",Negative,Customer Service,2018-06,0
3194,RDC,Nickel,5,2018-06-18 09:09:01,"Nickel, produit conforme, délai correct",Positive,Shipping,2018-06,1


In [84]:
#création du fichier csv global
df.to_csv('data_supply_chain.csv', index=False)

In [42]:
df_test_csv=pd.read_csv('data_supply_chain.csv')

In [44]:
df_test_csv.shape

(12669, 9)

In [46]:
#séparation en 2 df avec 1 df lignes ok et 1 df lignes sentiment + et rating <=2
df_filtre_ko=df.loc[(df['sentiment']=='Positive') & (df['rating']<=2)]

In [72]:
df_filtre_ko.shape
#917 lignes ko

(917, 9)

In [54]:
df_filtre_ok=df.loc[(df['sentiment']!='Positive') | (df['rating']>2)]

In [64]:
df_filtre_ok.shape

(11752, 9)

In [68]:
#on positionne le sentiment à 'Negative' pour toutes les lignes
df_filtre_ko['sentiment'] = fill_list(len(df_filtre_ko), 'Negative')

C:\Users\user\AppData\Local\Temp\ipykernel_10572\2038110268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtre_ko['sentiment'] = fill_list(len(df_filtre_ko), 'Negative')


In [70]:
df_filtre_ko['sentiment'].value_counts()

sentiment
Negative    917
Name: count, dtype: int64

In [74]:
#concaténation df_filtre_ok et df_filtre_ko
df_w_sentiment = pd.concat([df_filtre_ok, df_filtre_ko], axis=0)

In [76]:
df_w_sentiment.shape

(12669, 9)

In [78]:
df_w_sentiment['sentiment'].value_counts()

sentiment
Positive    5892
Neutral     4673
Negative    2104
Name: count, dtype: int64

In [80]:
df['sentiment'].value_counts()

sentiment
Positive    6809
Neutral     4673
Negative    1187
Name: count, dtype: int64

In [86]:
#création du fichier csv global avec retravail du sentiment
df_w_sentiment.to_csv('data_supply_chain_w_sentiment.csv', index=False)

In [88]:
df_test_csv=pd.read_csv('data_supply_chain.csv')
df_test_csv_w=pd.read_csv('data_supply_chain_w_sentiment.csv')

In [90]:
df_test_csv_w['sentiment'].value_counts()

sentiment
Positive    5892
Neutral     4673
Negative    2104
Name: count, dtype: int64

In [92]:
df_test_csv['sentiment'].value_counts()

sentiment
Positive    6809
Neutral     4673
Negative    1187
Name: count, dtype: int64